In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words
nltk.download('words')
import re

#Set Working Directory
os.chdir(r"/Users/glinn/Documents/CSCI5622-machine-learning/project/csci5622project")

[nltk_data] Downloading package words to /Users/glinn/nltk_data...
[nltk_data]   Package words is already up-to-date!


# Import Data sets, after splitting into 75,10,15 train, development, test. 

In [2]:
# https://stackoverflow.com/questions/50781562/stratified-splitting-of-pandas-dataframe-in-training-validation-and-test-set
# this should be 75%, 10%, 15%
test_full = pd.read_csv(r"data/2018-test.csv").dropna()
train_full = pd.read_csv(r"data/2018-train.csv").dropna()
validate_full = pd.read_csv(r"data/2018-validate.csv").dropna()# Clean the Data

# Clean the Data

We agreed to define success as simply pledged >= Goal, and to drop projects with a duration under 1 day, considered live at the time of acquiring the data, and projects that were cancelled. 

In [3]:
#Organize Data by deadline, launched, success, and duraction
def preprocess_df(df):
    df["deadline"] = pd.to_datetime(df["deadline"])
    df["launched"] = pd.to_datetime(df["launched"])
    #Define Success as pledged>=goal
    df["success"] = df["pledged"] >= df["goal"]
    df["duration"] = df["deadline"] - df["launched"]
    
    return df
#This drops data with a duration that is too short, live, or cancelled.
def clean_df(df):
    df = df.drop(df.loc[df["duration"] < datetime.timedelta(days=1)].index)
    df = df.drop(df.loc[df["state"] == "live"].index)
    df = df.drop(df.loc[df["state"] == "canceled"].index)
    
    return df

## Now apply these functions to the datasets:
test_organized = preprocess_df(test_full)
train_organized = preprocess_df(train_full)
validate_organized = preprocess_df(validate_full)
test = clean_df(test_organized)
train = clean_df(train_organized)
validate = clean_df(validate_organized)

# We want a function that turns the name into basic features

In [4]:
def WordsList(Text):
    wordList = re.sub("[^\w]", " ",  Text).split()
    return wordList

def unusual_words(text):
    Words = WordsList(text)
    text_vocab = set(w.lower() for w in Words if w.isalpha())
    english_vocab = set(w.lower() for w in nltk.corpus.words.words())
    unusual = text_vocab - english_vocab
    return len(sorted(unusual))

def ExtractFeatures(Name):
    NumChars = len(Name)
    NumWeirdChars = sum(not c.isalnum() for c in Name) - sum(c.isspace() for c in Name)
    NumUnusualWords = unusual_words(Name)
    return NumChars, NumWeirdChars,NumUnusualWords

def AddFeaturesTodf(df):
    NumElements = len(df["name"])
    print(NumElements)
    NumCharsList = []
    NumWeirdCharsList = []
    NumUnusualWordsList = []
    count=0
    items=0
    while items<NumElements:
        #This try simply skips over empty name data points
        try:
            NumChars, NumWeirdChars,NumUnusualWords = ExtractFeatures(df["name"][items+count])
            NumCharsList.append(NumChars)
            NumWeirdCharsList.append(NumWeirdChars)
            NumUnusualWordsList.append(NumUnusualWords)
            items+=1
        except:
            count+=1
    df["number of characters"] = NumCharsList
    df["number of weird characters"] = NumWeirdCharsList
    df["number of unusual words"] = NumUnusualWordsList
    return df

In [ ]:
validate_BasicTitleFeatures = AddFeaturesTodf(validate)

33249


In [5]:
validate["Number of characters"] = validate["name"].apply(lambda x: len(x))

In [ ]:
train_BasicTitleFeatures = AddFeaturesTodf(train)

In [ ]:
test_BasicTitleFeatures = AddFeaturesTodf(test)

In [ ]:
validate_BasicTitleFeatures.to_csv("data/2018-validate_BasicTitleFeatures.csv")
train_BasicTitleFeatures.to_csv("data/2018-train_BasicTitleFeatures.csv")
test_BasicTitleFeatures.to_csv("data/2018-test_BasicTitleFeatures.csv")

In [21]:
print(validate_BasicTitleFeatures)

   Unnamed: 0          ID                                      name  \
0      188874  1961049069                               Coming Home   
1      121497  1617129999  japanesepaper.ca , washi at a fair price   
2      314415   671311286       Tifany Lee & Co. Recording Campaign   

         category main_category currency   deadline     goal  \
0       Video Art           Art      USD 2015-05-18  15000.0   
1          Crafts        Crafts      CAD 2015-04-15    100.0   
2  Country & Folk         Music      USD 2011-07-22   2695.0   

             launched  pledged  ... backers  country usd pledged  \
0 2015-04-18 19:46:11      0.0  ...       0       US        0.00   
1 2015-02-14 23:25:36    263.0  ...       5       CA      210.03   
2 2011-06-22 23:43:48   2710.0  ...      20       US     2710.00   

   usd_pledged_real  usd_goal_real  success         duration  \
0              0.00       15000.00    False 29 days 04:13:49   
1            215.73          82.03     True 59 days 00:34

In [23]:
print(validate.iloc[0:3]["name"])

0                                 Coming Home
1    japanesepaper.ca , washi at a fair price
2         Tifany Lee & Co. Recording Campaign
Name: name, dtype: object


In [12]:
print(ExtractFeatures(validate["name"][0]))

(11, 0, 0)


In [6]:
print(validate["Number of characters"])

0        11
1        40
2        35
3        23
4         9
         ..
37860    24
37861    49
37862    27
37864    39
37865    50
Name: Number of characters, Length: 33249, dtype: int64
